In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate

In [2]:
root_data_dir = "../Datasets/NER/BiodivNER/"

dataset = "train"
train_csv_file_path = "train.csv"
val_csv_file_path = "dev.csv"
test_csv_file_path = "test.csv"

In [3]:
def loadData(csv_file_path):
  dataset_path = os.path.join(root_data_dir, csv_file_path)
  data = pd.read_csv(dataset_path, encoding="latin1")
  data = data.fillna(method="ffill")
  return data

In [4]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),                                                          
                                                        s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [5]:
data = loadData(train_csv_file_path)

In [6]:
val_data = loadData(val_csv_file_path)

In [7]:
test_data = loadData(test_csv_file_path)

In [8]:
VOCAB = list(set(list(data["Word"].values) + \
                 list(val_data["Word"].values) + \
                 list(test_data["Word"].values)))
VOCAB.append("ENDPAD")

n_words = len(VOCAB) #n_words includes all vocab from train and validation test.

tags = list(set(data["Tag"].values))

n_tags = len(tags)

In [9]:
getter = SentenceGetter(data)
sentences = getter.sentences
sent = getter.get_next()
print(sent)

[('bottom', 'O'), ('board', 'O'), ('debris', 'O'), (',', 'O'), ('frames', 'O'), (',', 'O'), ('landing', 'O'), ('platforms', 'O'), (')', 'O'), (',', 'O'), ('and', 'O'), ('isolates', 'O'), ('of', 'O'), ('microbes', 'O'), (',', 'O'), ('parasites', 'O'), ('and', 'O'), ('pathogens', 'O'), ('from', 'O'), ('honey', 'B-Organism'), ('bees', 'I-Organism'), ('.', 'O')]


In [10]:
getter_val = SentenceGetter(val_data)
sentences_val = getter_val.sentences
sent_val = getter_val.get_next()
print(sent_val)

[('However', 'O'), (',', 'O'), ('the', 'O'), ('lack', 'O'), ('of', 'O'), ('correlation', 'O'), ('between', 'O'), ('dung', 'O'), ('beetle', 'O'), ('community', 'B-Environment'), ('characteristics', 'O'), ('and', 'O'), ('dung', 'O'), ('removal', 'O'), ('highlights', 'O'), ('the', 'O'), ('need', 'O'), ('for', 'O'), ('further', 'O'), ('research', 'O'), ('into', 'O'), ('spatial', 'O'), ('variation', 'O'), ('in', 'O'), ('biodiversityÃ\x83Â¢Ã¢Â\x82Â¬Ã¢Â\x80Â\x9cecosystem', 'O'), ('function', 'O'), ('relationships', 'O'), ('and', 'O'), ('how', 'O'), ('the', 'O'), ('results', 'O'), ('of', 'O'), ('such', 'O'), ('studies', 'O'), ('are', 'O'), ('affected', 'O'), ('by', 'O'), ('methodological', 'O'), ('choices', 'O'), ('.', 'O')]


In [11]:
getter_test = SentenceGetter(test_data)
sentences_test = getter_test.sentences
sent_test = getter_test.get_next()
print(sent_test)

[('The', 'O'), ('primacy', 'O'), ('of', 'O'), ('either', 'O'), ('species', 'B-Quality'), ('or', 'O'), ('functional', 'O'), ('group', 'O'), ('richness', 'B-Quality'), ('effects', 'O'), ('depended', 'O'), ('on', 'O'), ('the', 'O'), ('sequence', 'O'), ('of', 'O'), ('testing', 'O'), ('these', 'O'), ('terms', 'O'), (',', 'O'), ('indicating', 'O'), ('that', 'O'), ('both', 'O'), ('aspects', 'O'), ('of', 'O'), ('richness', 'B-Quality'), ('were', 'O'), ('congruent', 'O'), ('and', 'O'), ('complementary', 'O'), ('to', 'O'), ('expected', 'O'), ('strong', 'O'), ('effects', 'O'), ('of', 'O'), ('legume', 'O'), ('presence', 'O'), ('and', 'O'), ('grass', 'B-Organism'), ('presence', 'O'), ('on', 'O'), ('plant', 'B-Organism'), ('chemical', 'B-Quality'), ('composition', 'I-Quality'), ('.', 'O')]


In [12]:
tag2id = {tag: id for id, tag in enumerate(tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [13]:
tag2id_list = list(tag2id.items())

In [14]:
id2tag_list = list(id2tag.items())

In [15]:
def get_text_tags_lists(sentences):
  texts = []
  tags = []
  for sent in sentences: #list of tuples    
    sent_texts = []
    sent_tags = []  
    for tuple1 in sent:  
      sent_texts.append(tuple1[0])
      sent_tags.append(tuple1[1])

    texts.append(sent_texts)
    tags.append(sent_tags)
  return texts, tags

In [16]:
train_texts, train_tags = get_text_tags_lists(sentences)
val_texts, val_tags = get_text_tags_lists(sentences_val)
test_texts, test_tags = get_text_tags_lists(sentences_test)

In [17]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5TokenizerFast, AdamW, Trainer, TrainingArguments
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report, accuracy_score

In [18]:
tokenizer = T5TokenizerFast.from_pretrained("t5-small")

In [19]:
label_all_tokens = False

In [20]:
def tokenize_and_align_labels(texts, tags):
    tokenized_inputs = tokenizer(texts, truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(tags):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [21]:
train_tokenized = tokenize_and_align_labels(train_texts, train_tags)
val_tokenized = tokenize_and_align_labels(val_texts, val_tags)
test_tokenized = tokenize_and_align_labels(test_texts, test_tags)

In [34]:
print(list(train_tokenized['input_ids']))
print(list(train_tokenized['labels']))

[[12474, 29, 52, 17700, 697, 29, 52, 276, 3171, 11392, 7678, 2926, 1054, 834, 7727, 725, 3128, 7, 485, 13516, 9651, 24231, 834, 345, 24231, 834, 517, 11339, 7, 834, 434, 23, 208, 11339, 7, 834, 308, 265, 11339, 7, 834, 2962, 9, 26, 26135, 834, 1409, 3318, 2754, 7, 834, 8952, 32, 162, 3318, 2754, 7, 834, 2703, 3216, 6792, 13, 13956, 7, 13, 8, 331, 15752, 16, 48, 17953, 17777, 5016, 3, 20275, 5, 3914, 3916, 3, 20056, 5, 2534, 4591, 2838, 5, 2577, 5373, 2838, 5, 1714, 27640, 3, 30345, 30345, 30345, 30345, 3, 30345, 30345, 30345, 30345, 3946, 302, 834, 1304, 3, 49, 89, 526, 972, 7155, 5072, 1881, 4267, 297, 3, 10, 17700, 697, 811, 5016, 3, 18, 1170, 11, 30253, 331, 818, 576, 22597, 16, 1475, 2597, 19, 4344, 5243, 57, 1951, 13, 3487, 11276, 16540, 53, 3, 6, 8, 3049, 1847, 18, 4302, 10361, 41, 446, 18, 254, 3, 61, 22455, 19, 46, 2433, 1295, 24, 65, 118, 14176, 38, 3, 9, 779, 19517, 8557, 13834, 306, 3244, 2354, 655, 1425, 3, 6, 16, 1090, 3, 6, 16, 10468, 5827, 10789, 7, 3, 5, 1], [2007, 1476

In [37]:
for i in train_tokenized['input_ids'][0:10]:
    print(len(i), i)

192 [12474, 29, 52, 17700, 697, 29, 52, 276, 3171, 11392, 7678, 2926, 1054, 834, 7727, 725, 3128, 7, 485, 13516, 9651, 24231, 834, 345, 24231, 834, 517, 11339, 7, 834, 434, 23, 208, 11339, 7, 834, 308, 265, 11339, 7, 834, 2962, 9, 26, 26135, 834, 1409, 3318, 2754, 7, 834, 8952, 32, 162, 3318, 2754, 7, 834, 2703, 3216, 6792, 13, 13956, 7, 13, 8, 331, 15752, 16, 48, 17953, 17777, 5016, 3, 20275, 5, 3914, 3916, 3, 20056, 5, 2534, 4591, 2838, 5, 2577, 5373, 2838, 5, 1714, 27640, 3, 30345, 30345, 30345, 30345, 3, 30345, 30345, 30345, 30345, 3946, 302, 834, 1304, 3, 49, 89, 526, 972, 7155, 5072, 1881, 4267, 297, 3, 10, 17700, 697, 811, 5016, 3, 18, 1170, 11, 30253, 331, 818, 576, 22597, 16, 1475, 2597, 19, 4344, 5243, 57, 1951, 13, 3487, 11276, 16540, 53, 3, 6, 8, 3049, 1847, 18, 4302, 10361, 41, 446, 18, 254, 3, 61, 22455, 19, 46, 2433, 1295, 24, 65, 118, 14176, 38, 3, 9, 779, 19517, 8557, 13834, 306, 3244, 2354, 655, 1425, 3, 6, 16, 1090, 3, 6, 16, 10468, 5827, 10789, 7, 3, 5, 1]
38 [2007,

In [38]:
for i in train_tokenized['labels'][0:10]:
    print(len(i), i)

192 ['O', -100, -100, 'O', -100, -100, -100, 'O', -100, 'O', 'O', 'O', -100, -100, -100, -100, 'B-Quality', -100, -100, 'B-Phenomena', 'B-Quality', 'O', -100, -100, 'O', -100, -100, 'O', -100, -100, -100, -100, -100, 'O', -100, -100, -100, -100, 'O', -100, -100, -100, -100, -100, 'O', -100, -100, 'O', -100, -100, -100, -100, -100, -100, 'O', -100, -100, -100, -100, -100, 'O', 'O', 'O', -100, 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', -100, -100, -100, -100, 'O', -100, -100, -100, -100, 'O', -100, -100, -100, 'O', -100, -100, -100, 'O', -100, -100, -100, -100, 'O', -100, -100, -100, -100, 'O', -100, -100, -100, 'O', -100, -100, -100, -100, 'O', 'B-Environment', 'O', -100, -100, 'O', -100, 'B-Organism', -100, 'O', 'O', 'O', -100, 'B-Phenomena', 'O', 'B-Quality', 'O', 'O', 'O', -100, 'O', 'B-Environment', 'I-Environment', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', -100, 'O', -100, 'O', 'O', -100, -100, -100, -100, 'O', 'O', -100, -100, 'O', -100, 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [39]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")
optimizer = AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

c:\Users\shann\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [40]:
training_args = TrainingArguments(
    output_dir="./output_dir",  # Change this to your desired output directory
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_steps=500,  # Save model every 500 steps
    evaluation_strategy="steps",
    eval_steps=250,  # Evaluate every 250 steps
    logging_steps=100,  # Log every 100 steps
    learning_rate=5e-5,
    save_total_limit=2,  # Only keep the last two models
    remove_unused_columns=False,  # Keep all columns in the dataset
    push_to_hub=False,  # Set to True if you want to push to the Hugging Face Model Hub
)

In [41]:
class NERDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [43]:
train_dataset = NERDataset(train_tokenized['input_ids'], train_tokenized['labels'])
val_dataset = NERDataset(val_tokenized['input_ids'], val_tokenized['labels'])
test_dataset = NERDataset(test_tokenized['input_ids'], test_tokenized['labels'])

In [44]:
model_name = 't5-small'
model = T5ForConditionalGeneration.from_pretrained(model_name, num_labels=len(tags))

In [52]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    # Create a long 1D list of y_true and y_pred
    y_true = []
    y_pred = []
    for preds, lbls in zip(predictions, labels):  
        [y_true.append(id2tag[l]) for p, l in zip(preds, lbls) if l != -100]
        [y_pred.append(id2tag[p]) for p, l in zip(preds, lbls) if l != -100]

    acc = accuracy_score([y_true], [y_pred])
    seqeval_report = classification_report([y_true], [y_pred])

    return {
        "accuracy": acc,
        "seqeval_report": seqeval_report
    }

In [47]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics
)

In [32]:
trainer.train()
eval_history = trainer.evaluate()
predictionsOutput = trainer.predict(test_dataset) 

print(eval_history["seqeval_report"])